# #230 Creep analysis of Tee beam
<i>Multispan tee beam from example #60 extended to demonstrate addition of creep and shrinkage analysis</i>
***

In [ ]:
# Input Params
spans_lengths = [8, 15, 8] #m
tee_section_width = 1.0
tee_section_depth = 1.0
tee_section_thk_flange = 0.2
tee_section_thk_web = 0.2

loading = 10

# Tendon profile variables
minz = -tee_section_depth*0.6
maxz =  tee_section_depth*0.2

Ec_shrt = 35e6
phi = 2
Ec_long = Ec_shrt/(1+phi)
approx_stress_at_transfer = 5e3 #kpa

#### Connect to LUSAS and create a new model database

In [ ]:
import sys; sys.path.append('../') # Reference modules in parent directory
from LPI import *
lusas = get_lusas_modeller()

from m100_Tools_And_Helpers import Helpers
Helpers.initialise(lusas)

if lusas.existsDatabase():
    raise Exception("This script will create a new model. Please save and close the current model and try again")

# Create a new model
lusas.newProject("Structural", "Creep Analysis of Tee Beam.mdl")
# Get the database
db = lusas.getDatabase()
# Set the analysis category & vertical axis
db.setAnalysisCategory("3D")
db.setVerticalDir("Z")
# Set the unit system
db.setModelUnits("kN,m,t,s,C")
db.setTimescaleUnits("Days") # specify age of concrete in days

# Create model attributes

In [ ]:
# Beam mesh with BMI21 (2 Noded Linear) Elements at 1m lengths
mesh_attr = db.createMeshLine("Beam Mesh").setSize("BMI21", 1)

In [ ]:
# Create a parametric section utility
util = db.createParametricSection('Tee Section Utility')
util.setType("T")
util.setDimensions(['B', 'D', 'tf', 'tw', 'r'], [tee_section_width, tee_section_depth, tee_section_thk_flange, tee_section_thk_web, 0.0])

# Now create the attribute which is based on the utility. The attribute provides additional transformations such as eccentricity and tapering
geom_section_attr = db.createGeometricLine('Tee Section')
geom_section_attr.setFromLibrary("Utilities", "", util.getName(), 0, 0, 0)

In [ ]:
def add_rebar(attr:IFReinforcementSection, row:int, faceIndex:int, layerIndex:int, barsCount:int, start:int, end:int, gap:float, dia:float):
    attr.setReinforcementValue("rebar", row, "faceIndex", faceIndex)
    attr.setReinforcementValue("rebar", row, "layerIndex", layerIndex)
    attr.setReinforcementValue("rebar", row, "barsCount", barsCount)
    attr.setReinforcementValue("rebar", row, "start", start)
    attr.setReinforcementValue("rebar", row, "end", end)
    attr.setReinforcementValue("rebar", row, "gap", gap)
    attr.setReinforcementValue("rebar", row, "barsDiameter", dia)
    attr.setReinforcementValue("rebar", row, "altBarsDiameter", -1.0)
    attr.setReinforcementValue("rebar", row, "endBarsDiameter", -1.0)
    attr.setReinforcementValue("rebar", row, "materialRef", "R1")

In [ ]:
# Create a Reinforcement attribute
attr = db.createReinforcementSection("RnfSct1")
attr.setGeometricAttribute(geom_section_attr)
#attr.setValue("calculateCrackWidths", True)
attr.setReinforcementValue("face", 0, "faceIndex", "All")
attr.setReinforcementValue("face", 0, "actualCover", 0.03)
attr.setReinforcementValue("face", 0, "linkAllowance", 0.0)
attr.setReinforcementValue("face", 0, "allowableCrackWidth", 0.0)
attr.setReinforcementValue("face", 0, "nominalCover", -1.0)

add_rebar(attr, row=0, faceIndex=1, layerIndex=1, barsCount=3, start=-1, end=-1, gap=0.0, dia=0.025)
add_rebar(attr, row=1, faceIndex=1, layerIndex=2, barsCount=2, start=-1, end=-1, gap=0.0, dia=0.025)
add_rebar(attr, row=2, faceIndex=5, layerIndex=1, barsCount=10, start=-1, end=-1, gap=0.0, dia=0.025)


rebar_line_attr = db.createReinforcementLine("RnfLn2")
rebar_line_attr.setSegmentValue("Longitudinal", 0, "reinforcementSection", "RnfSct1")
rebar_line_attr.setSegmentValue("Longitudinal", 0, "stretch", True)

# Link the geometric attribute with the associated reinforcement
geom_section_attr.setReinforcement(rebar_line_attr)


# Materials

A linear "RC Concrete" material will be created for the linear analysis which can also then be used for design checking.
For the creep analysis a creep material will be defined according to the eurocode creep and shrinkage equations

In [ ]:
concrete_values = {"Material":"Concrete", "Region":"Europe", "Standard":"EN1992-1-1:2004/2014", "Grade":"fck = 35MPa Quartzite",
                   "Advanced define":1, "RH":70.0, "K1":1.0, "IntFac":0.5, "Aggregate type":0.0, "Cement type":1.0}

concrete_material_attr = db.createIsotropicMaterial("Concrete", Ec_shrt, 0.2, 2.54842, 10.0E-6)
concrete_material_attr.setDefinitionMenuID(1, None, True)
concrete_material_attr.setDescription("fck = 35MPa Quartzite | Concrete | EN1992-1-1:2004/2014")

for name, value in concrete_values.items():
    concrete_material_attr.createValue(name)
    concrete_material_attr.setValue(name, value)

# Values that must be defined with dimensionality
concrete_material_attr.createValue("fck", 0, 1, -2, 0, 0, 0, 0)
concrete_material_attr.setValue("fck", 35.0E3)

In [ ]:
rebar_values = {"Material":"Steel - Reinforcing bar", "Region":"Europe", "Standard":"EN1992-1-1:2004/2014", "Grade":"500B", "epu":0.05}

rebar_material_attr = db.createIsotropicMaterial("Rebar", 200.0E6, 0.3, 7.85, 12.0E-6)
rebar_material_attr.setDefinitionMenuID(1, None, True)
rebar_material_attr.setDescription("500B | Steel - Reinforcing bar | EN1992-1-1:2004/2014")

for name, value in rebar_values.items():
    rebar_material_attr.createValue(name)
    rebar_material_attr.setValue(name, value)

# Values that must be defined with dimensionality
rebar_material_attr.createValue("fy", 0, 1, -2, 0, 0, 0, 0)
rebar_material_attr.setValue("fy", 500.0e3)

In [ ]:
material_attr = db.createCompoundMaterial("Mat3").setReinforcedConcrete()
material_attr.addMaterial(concrete_material_attr, "S1")
material_attr.addMaterial(rebar_material_attr, "R1")
material_attr.setActiveMaterial(0)

material_attr.setDefinitionMenuID(214, None , True)
material_attr.setDescription("RC Material - EN1992")
material_attr.createValue("Design code")
material_attr.setValue("Design code", "EN1992")

In [ ]:
# Creep & shrinkage material using the Eurocode creep equations
creep_material_attr = db.createIsotropicMaterial("Iso4", 36.75E6, 0.2, 2.5484)
creep_material_attr.addEurocodePlasticMaterial86(0.0, 0.0)
creep_material_attr.addEurocode2ConcreteCreep(48.0E3, "class N", 70.0, 0.0, False, 1.0, True, 1, 0.0)
creep_material_attr.addEurocode2ConcreteShrinkage(True)

In [ ]:
# Support attributes
fixedSupport = db.createSupportStructural("Fixed").setStructural("R", "R", "R", "F", "F", "F", "F", "F", "F")
pinnedSupport = db.createSupportStructural("Pinned").setStructural("F", "R", "R", "F", "F", "F", "F", "F", "F")

#### Create the model geometry

In [ ]:
''' Create deck lines'''
deck_lines:list[IFLine] = []

cur_x = 0
for length in spans_lengths:
    deck_lines.append(Helpers.create_line([cur_x, 0, 0], [cur_x + length, 0, 0]))
    # Increment the current x position
    cur_x += length


#### Assign attributes to the model geometry

In [ ]:
''' Assign the attributes to the deck_lines '''
# get the assignment object
assignment = lusas.assignment().setAllDefaults()
# Assign the mesh
mesh_attr.assignTo(deck_lines, assignment)
# Assign the geometry
geom_section_attr.assignTo(deck_lines, assignment)
# Assign the material
material_attr.assignTo(deck_lines, assignment)

''' Assign the supports to the points of the line '''
# Assign the fixed support to the first point
fixedSupport.assignTo(deck_lines[0].getStartPoint(), assignment)
# Assign the pinned support to the remaining points
pinnedSupport.assignTo([p.getEndPoint() for p in deck_lines], assignment)

In [ ]:
db.updateMesh()

# Loading

#### Create load cases and load attributes and assign the attributes to the model geometry

In [ ]:
db.getAnalysis("Analysis 1").setName("00 Base Analysis")

In [ ]:
# Get the automatically created loadcase in analysis 1 and add automatic gravity to it
# NOTE: getLoadset and setName function returns a reference to the IFLoadset baseclass and must be cast to IFLoadcase to access the addGravity function
gravity_loadcase = win32.CastTo(db.getLoadset("Loadcase 1", 0).setName("Gravity"), "IFLoadcase")
gravity_loadcase.addGravity(True)

In [ ]:
''' Create a beam load attribute '''
loadAttr = db.createLoadingBeamDistributed("UDL")
loadAttr.setBeamDistributed("Parametric", "Global", "beam")
loadAttr.addRow(0.0, 0.0, 0.0, -loading, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, -loading, 0.0, 0.0, 0.0)
# Assign the loading
loadAttr.assignTo(deck_lines, assignment)

# Create a loadcase for the Parapet loads to be applied to the entire deck surface
live_load_loadcases = []
for i in range(len(deck_lines)):
    live_load_loadcases.append(db.createLoadcase(f"Live Load {i+1}"))

# Assign live loads to each span
for i, loadcase in enumerate(live_load_loadcases):
    assign = lusas.assignment().setAllDefaults().setLoadset(loadcase)
    loadAttr.assignTo(deck_lines[i], assign)

### Creep Analysis

In [ ]:
def days_to_seconds(days:float)->float:
    return days * 24 * 60 *60

In [ ]:
creep_analysis = db.createAnalysisStructural(f"Creep Analysis", False)
# Add loadcase 
creep_loadcase = db.createLoadcase(f"Creep & Shrinkage", creep_analysis.getName())
creep_loadcase.addGravity(True) 

# Assign creep material
assignment = lusas.assignment().setAllDefaults().setLoadset(creep_loadcase)
creep_material_attr.assignTo(deck_lines, assignment)
# Assign 40% of the loading
assignment.setLoadFactor(0.4)
loadAttr.assignTo(deck_lines, assignment)

# Transient control with 50 time steps
control = creep_loadcase.setTransientControl(50).getTransientControl()
# Set default constant and output options
control.setConstants().setOutput()
# Loading is nonlinear
control.setNonlinearManual()
# Creep analysis is conducted in the time domain
# LPI inputs expected in seconds, regardless of global settings
control.setTimeDomainViscous(days_to_seconds(7))
control.setValue("TotalResponseTime", days_to_seconds(100*365))
# We want the time increments to increase later in the analysis
control.setValue("CEBFIP", 3.0)

In [ ]:
age_attr = db.createAge("Age - 7 days")
age_attr.setAgeType("specified")
age_attr.setAgeAtActivation(7)
age_attr.setAgeAtShrinkage(3)
age_attr.assignTo(deck_lines, assignment)

<H2>Solving the Analysis</H2>

In [ ]:
db.save()

for analysis in db.getAnalyses():
    analysis.solve(True)

db.openAllResults(True, True)

In [ ]:
# Display diagrams of bending moment
lusas.view().insertDiagramsLayer()
lusas.view().diagrams().setResults("Force/Moment - Thick 3D Beam", "My")
# Scale the view
lusas.view().setFromAxis("+X+Z")
lusas.view().setScaledToFit(True)

### Plot graphs of deflection over time

In [ ]:
# Create a graph of deflections for node 25

graphWizardObj = db.createGraphWizard("Creep Deflections Midspan")
graphWizardObj.setAutoUpdate(1)
# Loadcases to consider
graphWizardObj.setHistoryDump(creep_analysis.getName(), "None")
# X Axis
graphWizardObj.createNamedVariableHistory("Response time", False)
# Y Axis
graphWizardObj.createResultsHistoryNodal("Displacement", "RSLT", "Node", 25)